In [1]:
import os
import salobj
import SALPY_atcamera
import time
import numpy as np
import datetime
import nest_asyncio
nest_asyncio.apply()
import asyncio

/home/saluser/lsst_repos/salobj/python/salobj/test_utils.py:37: UserWarning: Could not import SALPY_Test; TestComponent will not work
  warnings.warn("Could not import SALPY_Test; TestComponent will not work")


In [2]:
import logging
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO,
                format='[%(asctime)s] [%(name)-12s:%(lineno)-4d] [%(levelname)-8s]: %(message)s',
                datefmt='%m-%d %H:%M:%S')

In [3]:
def getFileName():
    tmp_file='/tmp/atcamera_filename_current.dat'
    # This function assumes the file created in tmp_file
    # was written by this function. No error handling exists should the file be
    # modified or created via another method.

    def timeStamped(fname_suffix, fmt='AT-O-%Y%m%d-{fname:05}'):
        return (datetime.datetime.now().strftime(fmt).format(fname=fname_suffix))

    # today's format
    number=0 # assume zero for the moment
    file_date = timeStamped(number).split('-')[2]

    # Check to see if a file exists with a past filename
    if os.path.exists(tmp_file):
        # read in the file
        fh = open(tmp_file, 'r')
        first_line = (fh.readline())
        logging.info('Previous line in existing file {}'.format(first_line))

        # check to see if the date is the same
        logging.debug('file_date is: {}'.format(file_date))
        logging.debug('first_line is: {}'.format(first_line))
        if file_date in first_line:
            # grab file number and augment it
            old_num = first_line.split(',')[1]
            logging.debug('Previous Image number was: {}'.format(old_num))
            number = 1+int(old_num)
            logging.info('Incrementing from file to: {}'.format(number))
            fh.close()

        # Delete the file
        os.remove(tmp_file)

    # write a file with the new data
    fh = open(tmp_file, 'w')
    lines_of_text = [str(file_date)+','+str(number)]
    fh.writelines(lines_of_text)
    fh.close()

    fname = timeStamped(number)
    print('Newly generated filename: {}'.format(fname))
    return(fname)


In [8]:
class TakeImageStressTest:
    def __init__(self):
        self.atcamera = salobj.Remote(SALPY_atcamera, f'atcamera')
        
    async def takeImageLoop(self, nimages):

        for i in range(nimages):
            exposure = 1.+np.random.random()*5.
            atcamera_fname = getFileName()
            take_image_topic = self.atcamera.cmd_takeImages.DataType()
            take_image_topic.numImages = 1
            take_image_topic.expTime = exposure
            take_image_topic.shutter = False
            take_image_topic.imageSequenceName = str(atcamera_fname)
            take_image_topic.science = False

            end_readout = self.atcamera.evt_endReadout.next(flush=True, 
                                                            timeout=exposure+30.)
            take_image_task = self.atcamera.cmd_takeImages.start(take_image_topic)

            image = await asyncio.gather(end_readout, take_image_task)

            logging.info('[%04i/04%i] %s (%+03i, %04i, %s)', 
                         i+1, nimages,
                         image[0].imageName,
                         image[1].ack,
                         image[1].error,
                         image[1].result)



In [9]:
seq = TakeImageStressTest()

In [6]:
loop = asyncio.get_event_loop()

In [11]:
print('Start')
loop.run_until_complete(seq.takeImageLoop(1))
print('Done')

[10-02 18:27:46] [root        :19  ] [INFO    ]: Previous line in existing file 20181002,50
[10-02 18:27:46] [root        :29  ] [INFO    ]: Incrementing from file to: 51


Start
Newly generated filename: AT-O-20181002-00051


[10-02 18:27:57] [root        :28  ] [INFO    ]: [0001/041] AT-O-20181002-00051 (+303, 0000, Done : OK)


Done
